In [1]:
import requests
import json
from nltk import util
import markovify.text
import io

C:\Users\bram\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
%run keys.py
# bevat client_id en client_secret

# De comments binnenhalen

In [3]:
baseUrl = "https://graph.facebook.com"
pageName = "hln.be"

In [4]:
oauth = requests.get(baseUrl + "/oauth/access_token", 
                     params={
                        "client_id": client_id,
                        "client_secret": client_secret,
                        "grant_type": "client_credentials"})
access_token = oauth.text.split("=")[1]

## De laatste posts ophalen

In [ ]:
posts = requests.get(baseUrl + "/v2.5/ " + pageName + "/posts",
                     params={"access_token": access_token})
postids = []

for i in range(100): # vraag 100 keer voor "meer" posts
    postids.extend(list(post["id"] for post in json.loads(posts.text)["data"]))
    try:
        posts = requests.get(json.loads(posts.text)["paging"]["next"])
    except KeyError:
        break;

## De comments op die posts downloaden

In [ ]:
commentslist = []
for postid in postids:
    comments = requests.get(baseUrl + "/v2.5/" + postid + "/comments",
                            params={"access_token": access_token})

    while True:
        commentsobject = json.loads(comments.text)
        commentslist.extend(list(comment["message"] for comment in commentsobject["data"]))
        
        if not "paging" in commentsobject.keys():
            break
        
        comments = requests.get(baseUrl + "/v2.5/" + postid + "/comments",
                                params={
                                    "access_token": access_token,
                                    "after": commentsobject["paging"]["cursors"]["after"]})

Hoeveel comments hebben we nu?

In [ ]:
len(commentslist)

En hoe zien die comments er uit?

In [ ]:
commentslist[:50]

## Een back-up maken van de comments die we hebben gedownload

In [ ]:
with open('commentslist.json', 'w') as f:
    json.dump(commentslist, f)

## De back-up herstellen - begin hier om niet met de Facebook API te werken

In [ ]:
with open('commentslist.json') as f:
    commentslist = json.load(f)

# Een natural language model maken

In [ ]:
text_model2 = markovify.text.NewlineText("\n".join(commentslist))

## Gerandomizeerde zinnen laten schrijven

## state size = 2

In [ ]:
for i in range(50):
    print(text_model2.make_short_sentence(140))

## state size = 3

In [ ]:
text_model3 = markovify.text.NewlineText("\n".join(commentslist), state_size=3)

In [ ]:
for i in range(50):
    print(text_model3.make_sentence())